此代码的tensorflow的版本为1.3.1,deepctr的版本为0.7.4，这个无法在tensorflow2以及最新的deepctr上运行

In [1]:
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier
from matplotlib import pyplot as plt
from tqdm import tqdm
import time
import gc
import numpy as np
from scipy.stats import entropy
from gensim.models import Word2Vec
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr.models import *
from deepctr.inputs import  SparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau,LearningRateScheduler
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import tensorflow as tf
import os

C:\Users\Ksama\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ksama\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ksama\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\K

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0" #指定GPU

In [3]:
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

In [ ]:
#相关参数
weights_path = './fibinet_base.h5'
learning_rate = 1e-3
batch = 8192*4
n_epoch=100
embedding_dim = 8 #embedding维度一般来说越大越好，但是维度越大跑起来越慢

In [4]:
%%time
train_df = pd.read_csv('../data/train_data.csv',sep='|')
train_df = train_df.sample(frac=0.5).reset_index(drop=True)
test_df = pd.read_csv('../data/test_data_A.csv',sep='|')
df = pd.concat([train_df,test_df],axis=0)
test_id = test_df['id'].copy().reset_index(drop=True)

Wall time: 2min 9s


In [7]:
%%time
df=df.replace([np.inf, -np.inf],0)
df=df.fillna(0)

Wall time: 9.6 s


In [15]:
#处理类别特征
cate_cols = ['city_rank','creat_type_cd','dev_id','device_size','gender','indu_name','inter_type_cd','residence','slot_id','net_type','task_id','adv_id','adv_prim_id','age','app_first_class','app_second_class','career','city','consume_purchase','uid','dev_id','tags']

for f in tqdm(cate_cols):
    map_dict = dict(zip(df[f].unique(), range(df[f].nunique())))
    df[f] = df[f].map(map_dict).fillna(-1).astype('int32')
    df[f + '_count'] = df[f].map(df[f].value_counts())
df = reduce_mem(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:55<00:00,  2.53s/it]


-- Mem. usage decreased to 3287.04 Mb (57.7% reduction),time spend:0.16 min


In [16]:
drop_fea = ['pt_d','label','communication_onlinerate','index','uid','dev_id']
feature= [x for x in df.columns if x not in drop_fea]
print(len(feature))
print(feature)

53
['task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags', 'app_first_class', 'app_second_class', 'age', 'city', 'city_rank', 'device_name', 'device_size', 'career', 'gender', 'net_type', 'residence', 'his_app_size', 'his_on_shelf_time', 'app_score', 'emui_dev', 'list_time', 'device_price', 'up_life_duration', 'up_membership_grade', 'membership_life_duration', 'consume_purchase', 'communication_avgonline_30d', 'indu_name', 'id', 'city_rank_count', 'creat_type_cd_count', 'dev_id_count', 'device_size_count', 'gender_count', 'indu_name_count', 'inter_type_cd_count', 'residence_count', 'slot_id_count', 'net_type_count', 'task_id_count', 'adv_id_count', 'adv_prim_id_count', 'age_count', 'app_first_class_count', 'app_second_class_count', 'career_count', 'city_count', 'consume_purchase_count', 'uid_count', 'tags_count']


In [17]:
sparse_features = cate_cols
dense_features = [x for x in df.columns if x not in drop_fea+cate_cols] #这里的dense_feature可以把树模型的特征加进来
print('sparse_feature: {}'.format(sparse_features))
print('dense_feature: {}'.format(dense_features))

sparse_feature: ['city_rank', 'creat_type_cd', 'dev_id', 'device_size', 'gender', 'indu_name', 'inter_type_cd', 'residence', 'slot_id', 'net_type', 'task_id', 'adv_id', 'adv_prim_id', 'age', 'app_first_class', 'app_second_class', 'career', 'city', 'consume_purchase', 'uid', 'dev_id', 'tags']
dense_feature: ['spread_app_id', 'device_name', 'his_app_size', 'his_on_shelf_time', 'app_score', 'emui_dev', 'list_time', 'device_price', 'up_life_duration', 'up_membership_grade', 'membership_life_duration', 'communication_avgonline_30d', 'id', 'city_rank_count', 'creat_type_cd_count', 'dev_id_count', 'device_size_count', 'gender_count', 'indu_name_count', 'inter_type_cd_count', 'residence_count', 'slot_id_count', 'net_type_count', 'task_id_count', 'adv_id_count', 'adv_prim_id_count', 'age_count', 'app_first_class_count', 'app_second_class_count', 'career_count', 'city_count', 'consume_purchase_count', 'uid_count', 'tags_count']


In [18]:
#对dense_features进行归一化
mms = MinMaxScaler(feature_range=(0,1))
df[dense_features] = mms.fit_transform(df[dense_features])

Wall time: 3min 19s


In [19]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=df[feat].nunique(),embedding_dim=embedding_dim)
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features]

In [20]:
test_df = df[df["pt_d"]==8].copy().reset_index()
train_df = df[df["pt_d"]<8].reset_index()
del df
gc.collect()

0

In [24]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
online_train_model_input = {name:train_df[name].values for name in feature_names}
online_test_model_input = {name:test_df[name].values for name in feature_names}

In [23]:
def multi_category_focal_loss2(gamma=2., alpha=.25):
    """
    focal loss for multi category of multi label problem
    适用于多分类或多标签问题的focal loss
    alpha控制真值y_true为1/0时的权重
        1的权重为alpha, 0的权重为1-alpha
    当你的模型欠拟合，学习存在困难时，可以尝试适用本函数作为loss
    当模型过于激进(无论何时总是倾向于预测出1),尝试将alpha调小
    当模型过于惰性(无论何时总是倾向于预测出0,或是某一个固定的常数,说明没有学到有效特征)
        尝试将alpha调大,鼓励模型进行预测出1。
    Usage:
     model.compile(loss=[multi_category_focal_loss2(alpha=0.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    epsilon = 1.e-7
    gamma = float(gamma)
    alpha = tf.constant(alpha, dtype=tf.float32)

    def multi_category_focal_loss2_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        alpha_t = y_true * alpha + (tf.ones_like(y_true) - y_true) * (1 - alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1 - y_true, 1 - y_pred)
        ce = -tf.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), gamma)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss

    return multi_category_focal_loss2_fixed

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
plateau = ReduceLROnPlateau(monitor="val_auroc", verbose=1, mode='max', factor=0.3, patience=5)
early_stopping = EarlyStopping(monitor='val_auroc', patience=9, mode='max')
checkpoint = ModelCheckpoint(weights_path,
                             monitor='val_auroc',
                             verbose=0,
                             mode='max',
                             save_best_only=True)

model = FiBiNET(linear_feature_columns,dnn_feature_columns,task='binary',dnn_dropout=0.1,dnn_hidden_units=(512, 128),)

opt = Adam(lr=learning_rate)
model.compile(optimizer=opt,
              #loss='binary_crossentropy',
              loss = multi_category_focal_loss2(alpha=0.1, gamma=2),
              metrics=[auroc], )

history = model.fit(online_train_model_input, train_df['label'].values,
                    validation_split=0.2,
                    callbacks=[early_stopping,plateau,checkpoint],shuffle=True,
                    batch_size=batch, epochs=n_epoch)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


Train on 16762852 samples, validate on 4190714 samples
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Epoch 1/100
16762852/16762852 [==============================] - 307s 18us/sample - loss: 0.0122 - auroc: 0.6739 - val_loss: 0.0065 - val_auroc: 0.7866
Epoch 2/100
16762852/16762852 [==============================] - 258s 15us/sample - loss: 0.0065 - auroc: 0.7822 - val_loss: 0.0064 - val_auroc: 0.7993
Epoch 3/100
16762852/16762852 [==============================] - 255s 15us/sample - loss: 0.0065 - auroc: 0.7898 - val_loss: 0.0063 - val_auroc: 0.8039
Epoch 4/100
16762852/16762852 [==============================] - 258s 15us/sample - loss: 0.0065 - auroc: 0.7936 - val_loss: 0.0063 - val_auroc: 0.8064
Epoch 5/100
16762852/16762852 [==============================] - 262s 16us/sample - loss: 0.0065 - auroc: 0.7965 - val_loss: 0.0063 - val_auroc: 0.8075
Epoch 6/100
16762852/16762852 [==============================] - 261s 16us/sample - loss: 0.0065 - auroc: 0.7990 - val_loss: 0.0063 - val_auroc: 0.8080
Epoch 7/100
16762852/16762852 [==============================] - 261s 16us/sample - loss

In [ ]:
model.load_weights(weights_path)
y_pre = model.predict(online_test_model_input, batch_size=batch)

In [ ]:
res = pd.DataFrame()
res['id'] = test_id
res['probability'] = y_pre

In [ ]:
res.to_csv('./fibinet_base.csv',index=False)